In [1]:
!pip install opendatasets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.2/316.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/harunshimanto/epileptic-seizure-recognition/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  vaageeshad


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/harunshimanto/epileptic-seizure-recognition


In [7]:
import pandas as pd

df = pd.read_csv("epileptic-seizure-recognition/Epileptic Seizure Recognition.csv")

In [9]:
!pip install mne

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.2 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
import mne

eeg_data = df.iloc[:, :-1].values.T

n_channels = eeg_data.shape[0]
sfreq = 178
ch_names = [f"EEG {i+1}" for i in range(n_channels)]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types="eeg")

raw = mne.io.RawArray(eeg_data, info)

raw.plot(duration=5, n_channels=10, scalings="auto")

ValueError: could not convert string to float: 'X21.V1.791'